# Modern Data Cleaning : L'Agent IA (The Smart Way)

Dans le notebook précédent, nous avons tout fait à la main. C'était fastidieux.
Ici, nous allons construire un **Workflow Agentique** (Human-in-the-loop) pour automatiser le nettoyage.

**Le concept :**
1. **Analyse** : L'IA scanne le dataset et propose des actions.
2. **Validation** : L'humain sélectionne les actions pertinentes (Simulé ici).
3. **Exécution** : L'IA génère le code Python et l'exécute.

## 1. Setup & Configuration

In [ ]:
import pandas as pd
import google.generativeai as genai
import os
import json
import io
from dotenv import load_dotenv
import tabulate

# Nouveauté : Pydantic pour structurer la sortie
from pydantic import BaseModel, Field
from typing import List

# 1. Chargement des clés
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

# 2. Config Gemini
genai.configure(api_key=api_key)
model = genai.GenerativeModel('gemini-2.5-pro')

# 3. Chargement Data
df = pd.read_csv('dirty_sales_data.csv')
print("Données chargées :", df.shape)

In [ ]:
df.info()

In [ ]:
#buffer va permetter de transformer le .info en une string exploitable
buffer = io.StringIO()
df.info(buf=buffer)
info_str = buffer.getvalue()
print(info_str)
head_str = df.head(20).to_markdown()
print(head_str)

## 2. Phase 1 : Analyse Automatique

Nous avons deux façons de demander à l'IA :

### Approche V1 : La méthode naïve (Texte libre)
On demande simplement : "Analyse ça".

In [ ]:
prompt_v1 = f"""
Tu es un Data Analyst. Analyse ce dataset (info et head) et dis-moi ce qui ne va pas.
Sois concis.

DATA INFO:
{info_str}

DATA HEAD (Markdown):
{head_str}
"""

print(prompt_v1)

response_v1 = model.generate_content(prompt_v1)
print(response_v1.text)

In [ ]:
print(response_v1.text)

 **Problème** : C'est très lisible pour un humain, mais **impossible à utiliser dans une application** (comment créer des checkbox automatiquement à partir de ce texte ?).

### Approche V2 : La méthode industrielle (Structured Output)
On va définir un **Schéma de données** rigoureux avec `Pydantic` et forcer l'IA à le respecter.
C'est la méthode "Structured Output".

In [ ]:
# Définition de la structure attendue
class DataIssue(BaseModel):
    column: str = Field(..., description="Nom de la colonne concernée")
    issue_type: str = Field(..., description="Type de problème (ex: 'Format', 'Duplicata', 'Valeur manquante')")
    description: str = Field(..., description="Explication courte du problème")
    suggested_action: str = Field(..., description="Action Python suggérée (ex: 'pd.to_datetime', 'drop_duplicates')")

class DataAudit(BaseModel):
    issues: List[DataIssue]

# Appel à Gemini avec le schéma
prompt_v2 = f"""
Analyse ce dataset pour détecter les anomalies de qualité (types, doublons, formats).
Remplis la structure JSON demandée.

DATA INFO:
{info_str}

DATA HEAD:
{head_str}
"""

response_v2 = model.generate_content(
    prompt_v2,
    generation_config={"response_mime_type": "application/json", "response_schema": DataAudit}
)

# Parsing automatique
audit_result = json.loads(response_v2.text)
print(audit_result)

In [ ]:
issues_list = audit_result['issues']
# Affichage utilisable par le code
pd.DataFrame(issues_list)

 **Succès** : Nous avons maintenant une liste d'objets structurés que notre interface pourra manipuler.

## 3. Phase 2 : Simulation "Human-in-the-loop"

Dans l'app finale, l'utilisateur validera ces suggestions.
Ici, on garde tout.

In [ ]:
selected_actions = issues_list
print(f"Validation simulée de {len(selected_actions)} actions.")

selected_actions_restricted = selected_actions[0:3]
print(f"Validation simulée de {len(selected_actions_restricted)} actions.")

In [ ]:
selected_actions
selected_actions_restricted

## 4. Phase 3 : Génération & Exécution du Code

Dernière étape : transformer ces instructions JSON en code Python exécutable.

In [ ]:
# Définition de la structure de sortie pour le code
class CodeOutput(BaseModel):
    python_script: str = Field(..., description="Le script Python complet et exécutable, sans blocs markdown.")

def generate_cleaning_code(df_sample, actions):
    cols_context = df_sample.dtypes.to_dict()
    
    prompt_code = f"""
    Tu es un développeur Python Senior.
    Génère le code Python pour nettoyer le dataframe 'df' selon ces actions :
    {json.dumps(actions, indent=2)}
    df : {df_sample.head(10)}
    Colonnes : {cols_context}
    
    Règles :
    1. Code exécutable directement sur 'df'
    2. Gère les conversions avec robustesse.
    """
    
    # On force une sortie structurée contenant le code
    response = model.generate_content(
        prompt_code,
        generation_config={"response_mime_type": "application/json", "response_schema": CodeOutput}
    )
    
    # On récupère proprement le script dans le JSON
    return json.loads(response.text)['python_script']

cleaning_code = generate_cleaning_code(df, selected_actions)
print(cleaning_code)

In [ ]:
cleaning_code_restricted = generate_cleaning_code(df, selected_actions_restricted)
print(cleaning_code_restricted)

In [ ]:
# Exécution sécurisée (Scope local)
df_clean_agent = df.copy()
local_scope = {'df': df_clean_agent, 'pd': pd}

exec(cleaning_code, {}, local_scope)

df_final = local_scope['df']
df_final.head()

In [ ]:
# Exécution sécurisée (Scope local)
df_clean_agent_restricted = df.copy()
local_scope_restricted = {'df': df_clean_agent_restricted, 'pd': pd}

exec(cleaning_code_restricted, {}, local_scope_restricted)

df_final_restricted = local_scope_restricted['df']
df_final_restricted.head()

## 5. Le Grand Final : Tout en une fois !
Voici le script complet qui enchaîne tout : Analyse Robuste -> Génération Code -> Exécution.

In [ ]:
# === 1. ANALYSE ROBUSTE ===

# Prompt optimisé pour pousser l'IA dans ses retranchements
prompt_robust_audit = f"""
Tu es un Expert Data Engineer impitoyable sur la qualité de données.
Audite ce dataset pour produire un plan de nettoyage en béton.

DATA HEAD:
{head_str}

DATA INFO:
{info_str}

Instructions de nettoyage :
1. STANDARDISATION : Unifie la casse des textes (ex: Noms en Title Case, Catégories en minuscule...).
2. NETTOYAGE ROBUSTE : Pour les prix, utilise des Regex qui capturent tout (chiffres et séparateurs) et virent le reste.
3. LOGIQUE MÉTIER : Détecte les anomalies sémantiques (Quantités négatives ? Doublons d'ID ?).
4. DATES : Sois capable de parser des formats mixtes.

Retourne une liste structured (JSON) des actions à mener.
"""

print("🔍 Analyse approfondie en cours...")
response_audit = model.generate_content(
    prompt_robust_audit,
    generation_config={"response_mime_type": "application/json", "response_schema": DataAudit}
)
robust_issues = json.loads(response_audit.text)['issues']
print(f" {len(robust_issues)} problèmes détectés.")


# === 2. GENERATION CODE ===
print(" Génération du code de réparation...")
robust_code = generate_cleaning_code(df, robust_issues)
print("Code généré :")
print(robust_code)


# === 3. EXECUTION ===
print(" Exécution...")
df_clean_final = df.copy()
scope = {'df': df_clean_final, 'pd': pd}

try:
    exec(robust_code, {}, scope)
    df_final_result = scope['df']
    print("🎉 Terminé ! Aperçu avant/après :")
    display(df.head(3))
    display(df_final_result.head(3))
    print(df_final_result.info())
except Exception as e:
    print(" Erreur exécution :", e)